<a href="https://colab.research.google.com/github/nour3adel/R/blob/main/sub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Generalized Sequential Pattern (GSP) Mining

Apriori-based method: GSP (Generalized
Sequential Patterns: Srikant & Agrawal) 


<font color='cyan'> **Steps for Implementing GSP Mining** </font>

  Generate frequent events

 Repeat until no sequences can be generated:
> **1-** Generate candidate sequences.

> **2-** Candidate Pruning.

> **3-** Calculate support.

> **4-** Support pruning.




In [ ]:
import copy
import numpy as np
import pandas as pd
from operator import neg

### Dataset

| Seq. ID | Sequence |
|:--------------:|--------------:|
| 10 | <{bd}{c}{b}> |
| 20 | <{bf}{ce}{b}>|
| 30 | <{ag}{bf}> |
| 40 | <{be}{ce}> |
| 50 | <{a}{bd}{b}{c}{b}>|


In [ ]:
dataset = [   

    [['b','d'], ['c'], ['b']],  
    [['b' , 'f'], ['c','e'], ['b']], 
    [['a', 'g'], ['b','f']], 
    [['b','e'], ['c','e']],
    [['a'], ['b','d'],['b'],['c'],['b']]
]

### Definition of a Sequence

A sequence is an ordered list of elements
> s = < e1 e2 e3 … >

• Each element contains a collection of events
(items)
> ei = {i1, i2, …, ik}

• Length of a sequence, |s|, is given by the
number of elements in the sequence

• A k-sequence is a sequence that contains k
events (items)
5

***

### Subsequences
•Sequence t is a subsequence of s if each
ordered element in t is a subset of an
ordered element in s.

• Examples:

> <{1}{2}> is a subsequence of <{1}{2,3}{4}>

> <{1}{4}> is a subsequence of <{1}{2,3}{4}>

> But <{1}{2}{3}> is not a subsequence of <{1}{2,3}{4}>


Sequence A |Sequence B |Is B subsequence of A?
-----|-----|----- 
<{2,4} {3,5,6} {8}>|<{2} {3,6} {8}>|<font color='green'> **Yes** </font>
<{2,4} {3,5,6} {8}>|<{2} {8}>|<font color='green'> **Yes**  </font>
<{1,2} {3,4}>|<{1} {2}>|<font color='red'> **No**  </font>
<{2,4} {2,4} {2,5}>|<{2} {4}>|<font color='green'> **Yes**  </font>

## Sequence Length

In [ ]:
def sequence_length(sequence):
 
    return sum(len(i) for i in sequence)

In [ ]:
sequence_length([['a'], ['b', 'c'], ['a'], ['b', 'c', 'd']])

7

In [ ]:
def supports(sequence, cand_seq):
    for idx, event in enumerate(sequence):
        i = 0
        
        if set(event[1] if isinstance(event, tuple) else event).issuperset(cand_seq[i]):            
            i += 1
        
        # special case if cand_seq is a sequence of one element
        if i == len(cand_seq):
                return True
           

        for t, itemset in (sequence[idx + 1:] if isinstance(sequence[idx], tuple) 
                               else enumerate(sequence[idx + 1:], start=idx + 1)):

                
                if set(itemset).issuperset(cand_seq[i]):
                    i += 1

                # the sequence satisfies all the time constraints
                if i == len(cand_seq):
                    return True
    return False

In [ ]:
sequence = [[1, 3], [3, 4], [4], [5], [6, 7], [8]] 

In [ ]:
print(supports(sequence, [[3], [4]]))
print(supports(sequence, [[3], [6]]))
print(supports(sequence, [[1,9], [6]]))

True
True
False


# Support Count

In [ ]:
def count_support(dataset, cand_seq):
  
    return sum(1 for seq in dataset if supports(seq, cand_seq))
    

In [ ]:
count_support(dataset, [['b']])

5

In [ ]:
count_support(dataset, [['a'], ['b', 'c']])

0

 ### Generate candidate sequences.
 

 
    Generates one candidate of length k from two candidates of length (k-1)



In [ ]:
def gen_cands_for_pair(cand1, cand2):
    
    cand1_clone = copy.deepcopy(cand1)
    cand2_clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    
    if len(cand1[0]) == 1:
        cand1_clone.pop(0)
    else:
        cand1_clone[0] = cand1_clone[0][1:]
    # drop the rightmost item from cand2:
    if len(cand2[-1]) == 1:
        cand2_clone.pop(-1)
    else:
        cand2_clone[-1] = cand2_clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1_clone == cand2_clone:
        return []
    else:
        
        join = copy.deepcopy(cand1)
        if len(cand2[-1]) == 1:
            join.append(cand2[-1])
        else:
            join[-1].extend([cand2[-1][-1]])
        return join

In [ ]:
candA = [['a'], ['b', 'c'], ['d']]
candB = [['b', 'c'], ['d', 'e']]
gen_cands_for_pair(candA, candB)


[['a'], ['b', 'c'], ['d', 'e']]

In [ ]:
candA = [['a'], ['b', 'c'], ['d']]
candD = [['a'], ['b', 'c'], ['e']]
gen_cands_for_pair(candA, candD)

[]

 ### Generate candidate sequences.
 

 
    Generates the set of candidates of length k from the set of frequent sequences with length (k-1)

In [ ]:
def gen_cands(last_lvl_cands):
    
    k = sequence_length(last_lvl_cands[0]) + 1
    if k == 2:
        flat_short_cands = [item for sublist2 in last_lvl_cands for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flat_short_cands for b in flat_short_cands if b > a]
        result.extend([[[a], [b]] for a in flat_short_cands for b in flat_short_cands])
        return result
    else:
        cands = []
        for i in range(0, len(last_lvl_cands)):
            for j in range(0, len(last_lvl_cands)):
                new_cand = gen_cands_for_pair(last_lvl_cands[i], last_lvl_cands[j])
                if not new_cand == []:
                    cands.append(new_cand)
        cands.sort()
        return cands

Lets assume we know the frequent sequences of level 2:

In [ ]:
last_lvl_freq_patterns = [
    
    [['a', 'b']], 
    [['b', 'c']], 
    [['a'], ['b']], 
    [['a'], ['c']], 
    [['b'], ['c']], 
    [['c'], ['b']], 
    [['c'], ['c']]
]

Then we can compute the generate candidates for level 3:

In [ ]:
new_cands = gen_cands(last_lvl_freq_patterns)
new_cands

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['a', 'b', 'c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['b']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [ ]:
def gen_contiguous_direct_subsequences(sequence):
    
    result = []
    for i, itemset in enumerate(sequence):
        # first or last element
        if i == 0 or i == len(sequence) - 1:
            if len(itemset) == 1:
                seq_clone = copy.deepcopy(sequence)
                seq_clone.pop(i)
                result.append(seq_clone)
            else:
                for j in range(len(itemset)):
                    seq_clone = copy.deepcopy(sequence)
                    seq_clone[i].pop(j)
                    result.append(seq_clone)
        else:  # middle element
            if len(itemset) > 1:
                for j in range(len(itemset)):
                    seq_clone = copy.deepcopy(sequence)
                    seq_clone[i].pop(j)
                    result.append(seq_clone)
    return result

# Candidate Pruning.

In [ ]:
def prune_cands(last_lvl_cands, cands_gen):
       
    return [cand for cand in cands_gen if all(x in last_lvl_cands for x in (gen_contiguous_direct_subsequences(cand)))]

In [ ]:
cands_pruned = prune_cands(last_lvl_freq_patterns, new_cands)
cands_pruned

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [ ]:
min_sup = 0.4
def gsp(dataset, min_sup, verbose=False):
    
    overall = []
    min_sup *= len(dataset)
    # make the first pass over the sequence database to yield all the 1-element frequent subsequences
    items = sorted(set([item for sequence in dataset
                        for event in sequence
                        for item in (event[1] if isinstance(event, tuple) else event)]))
    single_item_sequences = [[[item]] for item in items]
    single_item_counts = [(s, count_support(dataset, s)) for s in single_item_sequences]
    single_item_counts = [(i, count) for i, count in single_item_counts if count >= min_sup]
    overall.append(single_item_counts)
    if verbose > 0:
        print('Result, lvl 1: ' + str(overall[0]))
        print('--------------------------------------------------------------------------------------------------')
    k = 1
    while overall[k - 1]:
        # 1. candidate generation: merge pairs of frequent subsequences found in the 
        # (k-1)th pass to generate candidate sequences that contain k items 
        last_lvl_cands = [x[0] for x in overall[k - 1]]
        cands_gen = gen_cands(last_lvl_cands)
        # 2. candidate pruning: prune candidate k-sequences that contain infrequent 
        # (contiguous) (k-1)-subsequences (Apriori principle)
        cands_pruned = prune_cands(last_lvl_cands, cands_gen)
        # 3. support counting: make a new pass over the sequence database to find 
        # the support for these candidate sequences
        cands_counts = [(s, count_support(dataset, s)) for s in cands_pruned]
        # 4. candidate elimination: eliminate candidate k-sequences whose actual 
        # support is less than `minsup`
        result_lvl = [(i, count) for i, count in cands_counts if count >= min_sup]
        if verbose > 1:
            
            print('Result, LeveL ' + str(k + 1) + ': ' + str(result_lvl))
            print('--------------------------------------------------------------------------------------------------')   
            if verbose > 1:
                print('Candidates generated, lvl ' + str(k + 1) + ': ' + str(cands_gen))
                print('\n') 
                print('Candidates pruned, lvl ' + str(k + 1) + ': ' + str(cands_pruned))
                
        overall.append(result_lvl)
        k += 1
    # "flatten" overall
    overall = overall[:-1]
    overall = [item for sublist in overall for item in sublist]
    overall.sort(key=lambda tup: (tup[1], neg(sequence_length(tup[0]))), reverse=True)
    return overall

In [ ]:
gsp(dataset, min_sup=0.4, verbose=2)

Result, lvl 1: [([['a']], 2), ([['b']], 5), ([['c']], 4), ([['d']], 2), ([['e']], 2), ([['f']], 2)]
--------------------------------------------------------------------------------------------------
Result, LeveL 2: [([['b', 'd']], 2), ([['b', 'f']], 2), ([['c', 'e']], 2), ([['a'], ['b']], 2), ([['b'], ['b']], 3), ([['b'], ['c']], 4), ([['b'], ['e']], 2), ([['c'], ['b']], 3), ([['d'], ['b']], 2), ([['d'], ['c']], 2)]
--------------------------------------------------------------------------------------------------
Candidates generated, lvl 2: [[['a', 'b']], [['a', 'c']], [['a', 'd']], [['a', 'e']], [['a', 'f']], [['b', 'c']], [['b', 'd']], [['b', 'e']], [['b', 'f']], [['c', 'd']], [['c', 'e']], [['c', 'f']], [['d', 'e']], [['d', 'f']], [['e', 'f']], [['a'], ['a']], [['a'], ['b']], [['a'], ['c']], [['a'], ['d']], [['a'], ['e']], [['a'], ['f']], [['b'], ['a']], [['b'], ['b']], [['b'], ['c']], [['b'], ['d']], [['b'], ['e']], [['b'], ['f']], [['c'], ['a']], [['c'], ['b']], [['c'], ['c']], 

[([['b']], 5),
 ([['c']], 4),
 ([['b'], ['c']], 4),
 ([['b'], ['b']], 3),
 ([['c'], ['b']], 3),
 ([['b'], ['c'], ['b']], 3),
 ([['a']], 2),
 ([['d']], 2),
 ([['e']], 2),
 ([['f']], 2),
 ([['b', 'd']], 2),
 ([['b', 'f']], 2),
 ([['c', 'e']], 2),
 ([['a'], ['b']], 2),
 ([['b'], ['e']], 2),
 ([['d'], ['b']], 2),
 ([['d'], ['c']], 2),
 ([['b'], ['c', 'e']], 2),
 ([['b', 'd'], ['b']], 2),
 ([['b', 'd'], ['c']], 2),
 ([['d'], ['c'], ['b']], 2),
 ([['b', 'd'], ['c'], ['b']], 2)]

### LeveL 1 (Frequent 1-sequences)

| Event | count |
|:------------:|-----------:|
| a | 2 |
| b | 5|
| c | 4 |
| d | 2|
| e | 2|
| f | 2|

### LeveL 2 (Frequent 2-sequences)

| Seq | count |
|:------------:|-----------:|
| {b}{b} | 3 |
| {a}{b} | 2|
| {b}{c}| 4 |
| {c}{b} | 3|
| {bd} | 2|
| {d}{b} | 2|
| {a}{b} | 2|
| {b}{e} | 2|
| {d}{c} | 2|
| {ce} | 2|

### LeveL 3 (Frequent 3-sequences)

| Event | count |
|:------------:|-----------:|
| {b}{c}{b} | 3 |
| {b}{ce} | 2|
| {bd}{b} | 2 |
| {bd}{c} | 2|
| {d}{c}{b} | 2|


### LeveL 4 (Frequent 4-sequences)

| Event | count |
|:------------:|-----------:|
| {bd}{c}{b} | 2 |

